In [1]:

import pandas as pd
import numpy as np
import model_evaluation_utils as meu
from sklearn.model_selection import train_test_split
import LoggerYN as YN

import time

import datetime

import pandas as pd
import numpy as np
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.metrics import *
import sklearn
from sklearn import linear_model
from sklearn import tree 
from sklearn import ensemble



In [2]:
dataset = pd.read_csv(r'2018.csv')

In [3]:
#Using only the following columns
df = dataset[["FL_DATE","OP_CARRIER","OP_CARRIER_FL_NUM",\
              "ORIGIN","DEST","CANCELLED","DISTANCE","CRS_DEP_TIME","CRS_ARR_TIME"]]

In [4]:
df.isnull().sum()

FL_DATE              0
OP_CARRIER           0
OP_CARRIER_FL_NUM    0
ORIGIN               0
DEST                 0
CANCELLED            0
DISTANCE             0
CRS_DEP_TIME         0
CRS_ARR_TIME         0
dtype: int64

In [5]:
#default datatypes
df.dtypes

FL_DATE               object
OP_CARRIER            object
OP_CARRIER_FL_NUM      int64
ORIGIN                object
DEST                  object
CANCELLED            float64
DISTANCE             float64
CRS_DEP_TIME           int64
CRS_ARR_TIME           int64
dtype: object

In [6]:
df.describe(include='all')

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CANCELLED,DISTANCE,CRS_DEP_TIME,CRS_ARR_TIME
count,7213446,7213446,7.213446e+06,7213446,7213446,7.213446e+06,7.213446e+06,7.213446e+06,7.213446e+06
unique,365,18,NaN,358,358,NaN,NaN,NaN,NaN
top,2018-11-25,WN,NaN,ATL,ATL,NaN,NaN,NaN,NaN
freq,22160,1352552,NaN,390046,390079,NaN,NaN,NaN,NaN
mean,NaN,NaN,2.607531e+03,NaN,NaN,1.616204e-02,7.999895e+02,1.329687e+03,1.486341e+03
std,NaN,NaN,1.860122e+03,NaN,NaN,1.260985e-01,5.981783e+02,4.909320e+02,5.183124e+02
min,NaN,NaN,1.000000e+00,NaN,NaN,0.000000e+00,3.100000e+01,1.000000e+00,1.000000e+00
25%,NaN,NaN,1.029000e+03,NaN,NaN,0.000000e+00,3.630000e+02,9.150000e+02,1.100000e+03
50%,NaN,NaN,2.131000e+03,NaN,NaN,0.000000e+00,6.320000e+02,1.320000e+03,1.515000e+03
75%,NaN,NaN,4.074000e+03,NaN,NaN,0.000000e+00,1.034000e+03,1.735000e+03,1.919000e+03


In [7]:
df.shape

(7213446, 9)

In [8]:
df["FL_DATE"] = pd.to_datetime(df["FL_DATE"]) 
df["Month"] = df["FL_DATE"].dt.month
df["Day"] = df["FL_DATE"].dt.day
df["Year"] = df["FL_DATE"].dt.year
df["Time_Hour"] = df["FL_DATE"].dt.hour
df["Time_Minute"] = df["FL_DATE"].dt.minute
df = df.drop("FL_DATE",axis=1)


/home/mubas/anaconda3/envs/bench/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/mubas/anaconda3/envs/bench/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/mubas/anaconda3/envs/bench/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [9]:
for column in (df.select_dtypes(include=['object']).columns):
    df[column] = df[column].astype('category').cat.codes

In [10]:
df.dtypes

OP_CARRIER              int8
OP_CARRIER_FL_NUM      int64
ORIGIN                 int16
DEST                   int16
CANCELLED            float64
DISTANCE             float64
CRS_DEP_TIME           int64
CRS_ARR_TIME           int64
Month                  int64
Day                    int64
Year                   int64
Time_Hour              int64
Time_Minute            int64
dtype: object

In [11]:
df

,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CANCELLED,DISTANCE,CRS_DEP_TIME,CRS_ARR_TIME,Month,Day,Year,Time_Hour,Time_Minute
0,13,2429,113,91,0.0,1605.0,1517,1745,1,1,2018,0,0
1,13,2427,186,307,0.0,414.0,1115,1254,1,1,2018,0,0
2,13,2426,320,91,0.0,846.0,1335,1649,1,1,2018,0,0
3,13,2425,293,247,0.0,1120.0,1546,1756,1,1,2018,0,0
4,13,2424,247,14,0.0,723.0,630,922,1,1,2018,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7213441,1,1815,90,71,0.0,331.0,1534,1714,12,31,2018,0,0
7213442,1,1816,71,92,0.0,936.0,1751,1952,12,31,2018,0,0
7213443,1,1817,71,214,0.0,511.0,2015,2107,12,31,2018,0,0
7213444,1,1818,71,283,0.0,130.0,1300,1350,12,31,2018,0,0


In [12]:
df.CANCELLED.value_counts

<bound method IndexOpsMixin.value_counts of 0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
7213441    0.0
7213442    0.0
7213443    0.0
7213444    0.0
7213445    0.0
Name: CANCELLED, Length: 7213446, dtype: float64>

In [13]:
X = df.drop('CANCELLED',axis=1)
y = df.pop('CANCELLED')


X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.80,random_state=0) 

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.svm import SVC


DT = DecisionTreeClassifier()
LR = LogisticRegression(max_iter=1000000)
RF = RandomForestClassifier(n_jobs=-1)
XGB = xgb.XGBClassifier(n_jobs=-1)
KNN = KNeighborsClassifier(n_jobs=-1)
class_labels = list(set(y))

In [22]:

# # Linear Model

memT,cpuT,gpuT = YN.StartLogger("Dataset_L_Instance_M","LinearModel_DT")
start = time.time()
DT.fit(X_train,y_train)
predictions = DT.predict(X_test)
meu.display_model_performance_metrics(true_labels=y_test, predicted_labels=predictions, classes=class_labels)
end = time.time()
YN.EndLogger(memT,cpuT,gpuT)
print("total time Linear Model Decision Tree DT ", str(datetime.timedelta(seconds=end-start)))

# # Linear Model

memT,cpuT,gpuT = YN.StartLogger("Dataset_L_Instance_M","LinearModel_LR")
start = time.time()
LR.fit(X_train,y_train)
predictions = LR.predict(X_test)
meu.display_model_performance_metrics(true_labels=y_test, predicted_labels=predictions, classes=class_labels)
end = time.time()
YN.EndLogger(memT,cpuT,gpuT)
print("total time Linear Model Logistic Regression", str(datetime.timedelta(seconds=end-start)))






Model Performance metrics:
------------------------------
Accuracy: 0.9762
Precision: 0.9775
Recall: 0.9762
F1 Score: 0.9768

Model Classification report:
------------------------------
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99   1419343
         1.0       0.29      0.32      0.31     23347

    accuracy                           0.98   1442690
   macro avg       0.64      0.66      0.65   1442690
weighted avg       0.98      0.98      0.98   1442690

total time Linear Model Decision Tree DT  0:02:06.554590
Model Performance metrics:
------------------------------
Accuracy: 0.9838


/home/mubas/anaconda3/envs/bench/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.9679
Recall: 0.9838
F1 Score: 0.9758

Model Classification report:
------------------------------


/home/mubas/anaconda3/envs/bench/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99   1419343
         1.0       0.00      0.00      0.00     23347

    accuracy                           0.98   1442690
   macro avg       0.49      0.50      0.50   1442690
weighted avg       0.97      0.98      0.98   1442690

total time Linear Model Logistic Regression 0:02:05.821501


In [23]:
# In[18]:
start = time.time()
memT,cpuT,gpuT = YN.StartLogger("Dataset_L_Instance_M","Non-LinearModel_RF")
RF.fit(X_train,y_train)
predictions = RF.predict(X_test)
meu.display_model_performance_metrics(true_labels=y_test, predicted_labels=predictions, classes=class_labels)
end = time.time()
YN.EndLogger(memT,cpuT,gpuT)
print("total time Non-Linear Model RF", str(datetime.timedelta(seconds=end-start)))

Model Performance metrics:
------------------------------
Accuracy: 0.9838
Precision: 0.9776
Recall: 0.9838
F1 Score: 0.9787

Model Classification report:
------------------------------
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99   1419343
         1.0       0.50      0.11      0.18     23347

    accuracy                           0.98   1442690
   macro avg       0.74      0.55      0.59   1442690
weighted avg       0.98      0.98      0.98   1442690

total time Non-Linear Model RF 0:22:52.710899


In [15]:
start = time.time()
memT,cpuT,_ = YN.StartLogger("Dataset_L_Instance_M","Non-LinearModel_XGB")
XGB.fit(X_train,y_train)
predictions = XGB.predict(X_test)
meu.display_model_performance_metrics(true_labels=y_test, predicted_labels=predictions, classes=class_labels)
end = time.time()
YN.EndLogger(memT,cpuT,_)
print("total time Non-Linear Model XGB", str(datetime.timedelta(seconds=end-start)))

[21:49:59] WARNING: /workspace/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Model Performance metrics:
------------------------------
Accuracy: 0.9846
Precision: 0.9824
Recall: 0.9846
F1 Score: 0.9779

Model Classification report:
------------------------------
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99   1419343
         1.0       0.84      0.06      0.11     23347

    accuracy                           0.98   1442690
   macro avg       0.91      0.53      0.55   1442690
weighted avg       0.98      0.98      0.98   1442690

total time Non-Linear Model XGB 0:36:54.391330


In [16]:
memT,cpuT,_ = YN.StartLogger("Dataset_S_Instance_S","LinearModel_KNN")
start = time.time()
KNN.fit(X_train,y_train)
predictions = KNN.predict(X_test)
meu.display_model_performance_metrics(true_labels=y_test, predicted_labels=predictions, classes=class_labels)
end = time.time()
YN.EndLogger(memT,cpuT,_)
print("total time Linear Model KNN", str(datetime.timedelta(seconds=end-start)))

Model Performance metrics:
------------------------------
Accuracy: 0.9834
Precision: 0.9727
Recall: 0.9834
F1 Score: 0.9761

Model Classification report:
------------------------------
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99   1419343
         1.0       0.28      0.02      0.03     23347

    accuracy                           0.98   1442690
   macro avg       0.63      0.51      0.51   1442690
weighted avg       0.97      0.98      0.98   1442690

total time Linear Model KNN 0:01:54.068293
